In [1]:
import torch

print(torch.__version__)

1.12.1+cu102


/home/sjkim/anaconda3/envs/STGTrans/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from torch.nn import Parameter
from torch_geometric.nn import ChebConv
from torch_geometric.nn.inits import glorot, zeros


class GCLSTM(torch.nn.Module):
    """
    Args:
        in_channels (int): Number of input features.
        out_channels (int): Number of output features.
        K (int): Chebyshev filter size :math:`K`.
        normalization (str, optional): The normalization scheme for the graph
            Laplacian (default: :obj:`"sym"`):

            1. :obj:`None`: No normalization
            :math:`\mathbf{L} = \mathbf{D} - \mathbf{A}`

            2. :obj:`"sym"`: Symmetric normalization
            :math:`\mathbf{L} = \mathbf{I} - \mathbf{D}^{-1/2} \mathbf{A}
            \mathbf{D}^{-1/2}`

            3. :obj:`"rw"`: Random-walk normalization
            :math:`\mathbf{L} = \mathbf{I} - \mathbf{D}^{-1} \mathbf{A}`

            You need to pass :obj:`lambda_max` to the :meth:`forward` method of
            this operator in case the normalization is non-symmetric.
            :obj:`\lambda_max` should be a :class:`torch.Tensor` of size
            :obj:`[num_graphs]` in a mini-batch scenario and a
            scalar/zero-dimensional tensor when operating on single graphs.
            You can pre-compute :obj:`lambda_max` via the
            :class:`torch_geometric.transforms.LaplacianLambdaMax` transform.
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
    """

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(GCLSTM, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.K = K
        self.normalization = normalization
        self.bias = bias
        self._create_parameters_and_layers()
        self._set_parameters()

    def _create_input_gate_parameters_and_layers(self):

        self.conv_i = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.W_i = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_i = Parameter(torch.Tensor(1, self.out_channels))

    def _create_forget_gate_parameters_and_layers(self):

        self.conv_f = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.W_f = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_f = Parameter(torch.Tensor(1, self.out_channels))

    def _create_cell_state_parameters_and_layers(self):

        self.conv_c = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.W_c = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_c = Parameter(torch.Tensor(1, self.out_channels))

    def _create_output_gate_parameters_and_layers(self):

        self.conv_o = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.W_o = Parameter(torch.Tensor(self.in_channels, self.out_channels))
        self.b_o = Parameter(torch.Tensor(1, self.out_channels))

    def _create_parameters_and_layers(self):
        self._create_input_gate_parameters_and_layers()
        self._create_forget_gate_parameters_and_layers()
        self._create_cell_state_parameters_and_layers()
        self._create_output_gate_parameters_and_layers()

    def _set_parameters(self):
        glorot(self.W_i)
        glorot(self.W_f)
        glorot(self.W_c)
        glorot(self.W_o)
        zeros(self.b_i)
        zeros(self.b_f)
        zeros(self.b_c)
        zeros(self.b_o)

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _set_cell_state(self, X, C):
        if C is None:
            C = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return C

    def _calculate_input_gate(self, X, edge_index, edge_weight, H, C, lambda_max):
        I = torch.matmul(X, self.W_i)
        I = I + self.conv_i(H, edge_index, edge_weight, lambda_max=lambda_max)
        I = I + self.b_i
        I = torch.sigmoid(I)
        return I

    def _calculate_forget_gate(self, X, edge_index, edge_weight, H, C, lambda_max):
        F = torch.matmul(X, self.W_f)
        F = F + self.conv_f(H, edge_index, edge_weight, lambda_max=lambda_max)
        F = F + self.b_f
        F = torch.sigmoid(F)
        return F

    def _calculate_cell_state(self, X, edge_index, edge_weight, H, C, I, F, lambda_max):
        T = torch.matmul(X, self.W_c)
        T = T + self.conv_c(H, edge_index, edge_weight, lambda_max=lambda_max)
        T = T + self.b_c
        T = torch.tanh(T)
        C = F * C + I * T
        return C

    def _calculate_output_gate(self, X, edge_index, edge_weight, H, C, lambda_max):
        O = torch.matmul(X, self.W_o)
        O = O + self.conv_o(H, edge_index, edge_weight, lambda_max=lambda_max)
        O = O + self.b_o
        O = torch.sigmoid(O)
        return O

    def _calculate_hidden_state(self, O, C):
        H = O * torch.tanh(C)
        return H
    
    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor = None,
        H: torch.FloatTensor = None,
        C: torch.FloatTensor = None,
        lambda_max: torch.Tensor = None,
    ) -> torch.FloatTensor:
        """
        Making a forward pass. If edge weights are not present the forward pass
        defaults to an unweighted graph. If the hidden state and cell state
        matrices are not present when the forward pass is called these are
        initialized with zeros.

        Arg types:
            * **X** *(PyTorch Float Tensor)* - Node features.
            * **edge_index** *(PyTorch Long Tensor)* - Graph edge indices.
            * **edge_weight** *(PyTorch Long Tensor, optional)* - Edge weight vector.
            * **H** *(PyTorch Float Tensor, optional)* - Hidden state matrix for all nodes.
            * **C** *(PyTorch Float Tensor, optional)* - Cell state matrix for all nodes.
            * **lambda_max** *(PyTorch Tensor, optional but mandatory if normalization is not sym)* - Largest eigenvalue of Laplacian.

        Return types:
            * **H** *(PyTorch Float Tensor)* - Hidden state matrix for all nodes.
            * **C** *(PyTorch Float Tensor)* - Cell state matrix for all nodes.
        """
        H = self._set_hidden_state(X, H)
        C = self._set_cell_state(X, C)
        I = self._calculate_input_gate(X, edge_index, edge_weight, H, C, lambda_max)
        F = self._calculate_forget_gate(X, edge_index, edge_weight, H, C, lambda_max)
        C = self._calculate_cell_state(X, edge_index, edge_weight, H, C, I, F, lambda_max)
        O = self._calculate_output_gate(X, edge_index, edge_weight, H, C, lambda_max)
        H = self._calculate_hidden_state(O, C)
        return H, C


In [8]:
import networkx as nx
import numpy as np

def create_mock_data(number_of_nodes, edge_per_node, in_channels):
    """
    Creating a mock feature matrix and edge index.
    """
    graph = nx.watts_strogatz_graph(number_of_nodes, edge_per_node, 0.5)
    edge_index = torch.LongTensor(np.array([edge for edge in graph.edges()]).T)
    X = torch.FloatTensor(np.random.uniform(-1, 1, (number_of_nodes, in_channels)))
    #X = torch.FloatTensor(np.resize(np.arange(number_of_nodes), (in_channels, number_of_nodes)).T)
    return X, edge_index

def create_mock_edge_weight(edge_index):
    """
    Creating a mock edge weight tensor.
    """
    return torch.FloatTensor(np.random.uniform(0, 1, (edge_index.shape[1])))

In [9]:
def k_hop_based_subgraph_generator_BFS(k, feature_vector, adjacency_matrix):
    N = len(feature_vector)        
    subgraphs = []

    def BFS(u):
        visit = [False] * N
        qu = [(u, 0)]
        visit[u] = True
        ret = []

        while len(qu) > 0:
            u, dist = qu.pop(0)
            if dist > k:
                break
            ret.append(u)

            for v in range(N):
                if adjacency_matrix[u][v] == 1 and visit[v] == False:
                    qu.append((v, dist + 1))
                    visit[v] = True
        return ret    
    
    for i in range(N):
        candidate_list = BFS(i)
        sub_feature = feature_vector[candidate_list]
        sub_adj = adjacency_matrix[:,candidate_list][candidate_list,:]
        subgraphs.append([sub_feature, sub_adj])
    return subgraphs

In [10]:
def tar_city_model():

    number_of_nodes = 4
    edge_per_node = 2
    
    in_channels = 64
    out_channels = 16
    K = 2
    k = 1
    number_of_layers = 5

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X, edge_index = create_mock_data(number_of_nodes, edge_per_node, in_channels)
    X = X.to(device)
    edge_index = edge_index.to(device)
    edge_weight = torch.FloatTensor(np.ones(edge_index.shape[1])).to(device)
    adj = torch.FloatTensor(np.zeros((number_of_nodes, number_of_nodes))).to(device)
    for i in range(edge_index.shape[1]):
        adj[edge_index[0][i]][edge_index[1][i]] = edge_weight[i]
        adj[edge_index[1][i]][edge_index[0][i]] = edge_weight[i]
    
    print(X)
    print(adj)

    subgraph = k_hop_based_subgraph_generator_BFS(k, X, adj)
    subGCLSTM = []

    for sub_feature, sub_adj in subgraph:
        layer = GCLSTM(in_channels=in_channels, out_channels=out_channels, K=K).to(device)

        edges = []
        for i in range(len(sub_adj)):
            for j in range(len(sub_adj[0])):
                if sub_adj[i][j] == 1:
                    edges.append([i, j])
        
        X = sub_feature
        edge_index = torch.LongTensor(np.array(edges).T).to(device)
        edge_weight = torch.FloatTensor(np.ones(edge_index.shape[1])).to(device)
        
        #print(X.shape)
        #print(edge_index.shape)
        #print(edge_weight.shape)

        for i in range(number_of_layers):
            if i == 0:
                H, C = layer(X, edge_index, edge_weight)
            else:
                H, C = layer(X, edge_index, edge_weight, H, C)
        
        #subGCLSTM.append([H, C])
        subGCLSTM.append(H[0])
    
    subGCLSTM = torch.stack(subGCLSTM, 0)
    
    return subGCLSTM

In [11]:
tar_city_model()

tensor([[ 2.9688e-01, -6.2442e-01, -8.4654e-01,  9.0288e-01, -1.2188e-01,
          1.0943e-01,  1.0685e-01,  6.3113e-01, -9.0448e-01, -7.2135e-01,
         -7.3216e-01,  9.5887e-01,  5.2013e-01,  6.9237e-01,  5.7520e-01,
          4.4226e-01,  7.3227e-01, -8.2984e-01,  1.3687e-01,  4.7502e-01,
         -9.9719e-01, -1.3444e-01, -6.2768e-01,  8.8830e-01,  3.3954e-01,
         -7.6304e-01,  5.0844e-01,  7.7361e-01, -1.8637e-02, -9.6821e-01,
          7.8430e-02,  9.0718e-01, -6.2693e-01, -2.4073e-01, -2.8049e-01,
          8.1025e-03,  4.9677e-02, -5.6954e-01,  9.1469e-01, -9.5053e-01,
          5.6249e-01,  6.1196e-01, -3.3739e-01, -9.5040e-01,  4.1490e-01,
          7.9384e-01,  1.6259e-01, -6.9614e-01,  1.2874e-01, -7.6601e-01,
          6.6441e-01, -6.5492e-01,  3.5661e-01, -2.4235e-04,  1.5453e-02,
          2.7323e-01,  5.5767e-01, -2.8122e-01,  7.6146e-01, -3.0145e-01,
         -1.5504e-01, -2.1270e-01,  4.5350e-01, -1.7934e-01],
        [-1.3760e-01, -1.6416e-01, -2.9669e-01, -3

tensor([[-0.3143,  0.3424,  0.4997,  0.4018,  0.2919,  0.2208,  0.3149, -0.1876,
          0.2231, -0.3326,  0.0305, -0.0267,  0.0080, -0.0601, -0.3850,  0.1907],
        [-0.1015,  0.0262, -0.7688,  0.2065,  0.0045,  0.1151, -0.3001,  0.2636,
          0.4097, -0.3077, -0.4377,  0.2022,  0.1087, -0.4507, -0.0086, -0.1611],
        [ 0.1511,  0.1041, -0.0163,  0.2887, -0.3903, -0.0236,  0.5719,  0.2249,
         -0.0613,  0.0103,  0.1467, -0.0134, -0.0782,  0.0528,  0.2347, -0.1186],
        [ 0.0158, -0.1025, -0.1960, -0.1387, -0.4104, -0.0669, -0.2548, -0.5697,
         -0.0173, -0.1488,  0.5804,  0.3054,  0.4924,  0.2643, -0.1180,  0.3707]],
       device='cuda:0', grad_fn=<StackBackward0>)

In [12]:
from tqdm import tqdm

model = tar_city_model()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(50)):
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = torch.mean((y_hat-snapshot.y)**2)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()

tensor([[ 0.7015,  0.7190,  0.8282, -0.4113,  0.7139, -0.8206,  0.9074, -0.5064,
         -0.6740, -0.7118, -0.0896,  0.6049,  0.1442, -0.2489,  0.4248,  0.1383,
         -0.8398, -0.5556,  0.9540,  0.8415,  0.0871, -0.5700, -0.9011,  0.0491,
          0.9962,  0.5597,  0.1744, -0.1462,  0.7769, -0.6330, -0.0785, -0.1797,
          0.7256, -0.6184,  0.5107, -0.4516,  0.1102, -0.7135,  0.0948,  0.6320,
          0.6777, -0.7451, -0.4012,  0.3777,  0.1393,  0.9869,  0.7218,  0.3166,
          0.2912,  0.4273,  0.3678, -0.8668,  0.4619,  0.6338, -0.9946, -0.4875,
          0.6419,  0.7407, -0.0352, -0.0258,  0.5778,  0.3292,  0.5178,  0.9417],
        [-0.5871,  0.8162, -0.6052, -0.0746, -0.5255,  0.6210,  0.2312,  0.9470,
         -0.8650,  0.3463,  0.1809,  0.1191, -0.7839,  0.1871,  0.8393,  0.4179,
         -0.8000, -0.7423,  0.1952, -0.3607,  0.0377,  0.7135,  0.6842, -0.4546,
          0.1563,  0.8178,  0.9238,  0.7610,  0.1599,  0.3040, -0.1780, -0.0716,
         -0.6596, -0.4401, 

AttributeError: 'Tensor' object has no attribute 'parameters'